In [1]:
# 

In [ ]:
from collections.abc import Iterable
from chemicals import *
import pandas as pd
import re
compound = ['CO2', 'N2', 'CH4', 'C2H6', 'C3H8', 'C4H10']
mole_fraction = [0.12, 0.021, 0.8, 0.042, 0.013, 0.004]
weight_fraction = [0.12, 0.021, 0.8, 0.042, 0.013, 0.004]
MOLAR_VOLUME = 379.3  #scf/lbmole

In [2]:
def convert_lbmole_CO2_to_lbCO2():
    '''This function converts lbmole CO2/time to lb CO2/time
    
    where:
        1 lbmole CO2 = 44lb CO2
    '''
    return 44

In [3]:
def convert_lb_to_tonnes():
    '''This function converts pounds to tonnes.
    
        where:
        1 tonnes = 2204.62lb
    '''
    return 1/2204.62

In [4]:
def CO2_emissions_inlet_known_volume_mole_basis(HCin, CFhc):
    '''
    This function calculates the Carbon dioxide emission rate in tonnes when the flare inlet data is known.
    
    Definition of variables:
    HCin = Flare hydrocarbon inlet rate (to the flare)
    CFhc = Carbon weight fraction in the inlet hydrocarbon
    FE = Flare combustion efficiency
    Mco2 = Mass of CO2 in flared stream based on CO2 composition of the inlet stream.
    
    CALCULATION STEPS:
    '''
    CO2_mass_emission_rate = HCin*CFhc*FUEL_EFFICIENCY*convert_lbmole_CO2_to_lbCO2()*convert_lb_to_tonnes() + mass_co2
    return CO2_mass_emission_rate

In [5]:

def convert_scf_to_lbmole(volume_rate):
    '''This function converts volumetric flowrate in scf/time to molar flowrate in lbmole/time
    Molar volume of a gas at Standard temperature and pressure is given as:
    1 lbmole gas = 379.3scf gas
    
    Arguments: volume_rate in scf/time  
    '''
    molar_flowrate = volume_rate*(1/MOLAR_VOLUME)
    return molar_flowrate

In [6]:
def molecular_weight_of_mixture_molefraction(mole_fraction:Iterable[float], molecular_weight:Iterable[float]):
    '''This function calculates molecular weight fraction of the gas mixture given:
    mole fraction, and molecular weight
    
    '''
    x = zip(mole_fraction, molecular_weight)
    product_list = []
    for a, b in x:
        product_list.append(a*b)
        
    return sum(product_list)
    
def molecular_weight_of_mixture_weightfraction(weight_fraction:Iterable[float], molecular_weight:Iterable[float]):
    '''This function calculates molecular weight fraction of the gas mixture given:
    component weight fractions, and molecular weights.
    
    '''
    x = zip(weight_fraction, molecular_weight)
    product_list = []
    for a, b in x:
        product_list.append(a/b)
        
    return sum(product_list)

In [ ]:
def molefraction_to_massfraction(mole_fraction:Iterable[float], molecular_weight:Iterable[float]):
    '''This function converts mole fraction to weight or mass fraction.
    Arguments: 
    1) mole fraction(Array)
    2) Molecular weight of each gas component in lb/lbmole
    
    '''
    x = zip(mole_fraction, molecular_weight)
    product_list = []
    for a, b in x:
        product_list.append(a*b/molecular_weight_of_mixture_molefraction(mole_fraction, molecular_weight))
    return product_list

def massfraction_to_molefraction(weight_fraction:Iterable[float], molecular_weight:Iterable[float]):
    '''This function converts mole fraction to weight or mass fraction.
    Arguments: 
    1) weight fraction(Array)
    2) Molecular weight of each gas component in lb/lbmole
    
    '''
    x = zip(weight_fraction, molecular_weight)
    product_list = []
    for a, b in x:
        product_list.append(a*molecular_weight_of_mixture_molefraction(mole_fraction, molecular_weight)/b)
    return product_list

In [ ]:
def get_molecular_weight_list(mixture):
    '''This function returns the list of molecular weight for the components in a gas mixture.
        The function Receives an array or list of the common name or molecular formular of the gas components as argument.
    '''
    # list of gas components.

    list_of_components = []
    list_of_molecular_weight = []
    list_of_molecular_formula = []
    for i in identifiers.pubchem_db:
        for j in mixture:
            if i.formula == j or i.common_name == j:
                list_of_components.append(i.common_name)
                list_of_molecular_weight.append(i.MW)
                list_of_molecular_formula.append(i.formula)
            else:
                None

    # Create a pandas dataframe from three lists of column names Compound, Formula, and Molecular weight.
    data = {
        'Compound': list_of_components,
        'Formula': list_of_molecular_formula,
        'Molecular Weight': list_of_molecular_weight
    }
    df = pd.DataFrame(data)

    return df

In [ ]:
def adjusted_composition(compound, composition):
    '''This function takes the mole fraction or weight fraction of the gas mixture as argument and returns a list of the adjusted mole fraction of weight fraction.'''

    # combine two lists, compound and composition into a dictionary of key and value pair.
    dictionary = dict(zip(compound, composition))
    included_list = find_hydrocarbons_in_gas_mixture(compound)
    for key, value in dictionary.items():
        if key in [i.formula for i in included_list]:
            dictionary[key] = value  # Set value to value if key is in include_list
        else:
            dictionary[key] = 0 # Set value to value if key is not in include_list

    # find the sum of the new dictionary values when non-hydrocarbon components are set to zero
    sum_composition = sum(dictionary.values())
    for key, values in dictionary.items():
        dictionary[key] = values/sum_composition
    return dictionary

In [ ]:
def carbon_ratio(compound):
    matches = re.findall(r"C(\d+)H", compound)
    if compound == 'CH4':
        digits = 1
    elif matches == []:
        digits = None
    else:
        digits = matches[0]
    return digits

In [ ]:
def adjusted_composition_and_molecular_weight_table(compound, composition):

    # List to define the sorting order
    sorting_order = list(adjusted_composition(compound, composition))

    # Sort the DataFrame based on the sorting_order list
    df_sorted = get_molecular_weight_list(compound).sort_values(by='Formula', key=lambda x: x.map({value: i for i, value in enumerate(sorting_order)})).reset_index(drop=True)

    # Append adjusted composition to sorted dataFrame
    df_sorted['Adjusted Composition'] = pd.Series(adjusted_composition(compound, composition).values())


    # Append Carbon Ratio to the dataframe
    df_sorted['Carbon Ratio'] = df_sorted['Formula'].apply(lambda formula: carbon_ratio(formula))
    return df_sorted.fillna(0)

In [ ]:
def carbon_content_mole_basis():
    data_table = adjusted_composition_and_molecular_weight_table(compound, mole_fraction)
    data_table['Mole Fraction'] = pd.Series(mole_fraction)
    data_table = data_table.fillna(0)
    mole_fraction_carbonratio = lambda row: row['Mole Fraction']*float(row['Carbon Ratio'])

    data_table['Carbon Content'] = data_table.apply(mole_fraction_carbonratio, axis=1)

    return data_table

In [ ]:
def carbon_content_weight_basis():
    data_table = adjusted_composition_and_molecular_weight_table(compound, weight_fraction)
    data_table['Weight Fraction'] = pd.Series(weight_fraction)
    data_table = data_table.fillna(0)
    mole_fraction_carbonratio = lambda row: row['Adjusted Composition']*float(row['Carbon Ratio'])*12/row['Molecular Weight']

    data_table['Carbon Content'] = data_table.apply(mole_fraction_carbonratio, axis=1)

    return data_table

In [ ]:
def carbon_content_fuel_mixture_mole_basis():
    C_mixture = sum(carbon_content_mole_basis()['Carbon Content'])
    return C_mixture

In [ ]:
def carbon_content_fuel_mixture_weight_basis():
    C_mixture = sum(carbon_content_weight_basis()['Carbon Content'])
    return C_mixture

In [ ]:
def mass_CO2_mole_composition_basis(compound, composition):
    '''Mass of CO2 in the flared stream based on CO2 composition of the inlet stream'''
    dictionary = dict(zip(compound, composition))
    co2_composition = dictionary['CO2']
    return co2_composition

In [ ]:
def convert_scf_to_lbmole(volume_rate):
    '''This function converts volumetric flowrate in scf/time to molar flowrate in lbmole/time
    Molar volume of a gas at Standard temperature and pressure is given as:
    1 lbmole gas = 379.3scf gas
    
    Arguments: volume_rate in scf/time  
    '''
    molar_flowrate = volume_rate*(1/MOLAR_VOLUME)
    return molar_flowrate

In [ ]:
def CO2_emissions_inlet_known_volume_mole_basis(HCin, FUEL_EFFICIENCY):
    '''
    This function calculates the Carbon dioxide emission rate in tonnes when the flare inlet data is known.
    
    Definition of variables:
    HCin = Flare hydrocarbon inlet rate (to the flare) in SCF
    CFhc = Carbon weight fraction in the inlet hydrocarbon
    FE = Flare combustion efficiency
    Mco2 = Mass of CO2 in flared stream based on CO2 composition of the inlet stream.
    
    CALCULATION STEPS:
    '''
    CO2_mass_emission_rate = convert_scf_to_lbmole(HCin)*(carbon_content_fuel_mixture_mole_basis()*FUEL_EFFICIENCY + mass_CO2_mole_composition_basis(compound, mole_fraction))*convert_lbmole_CO2_to_lbCO2()*convert_lb_to_tonnes()
    return CO2_mass_emission_rate

In [ ]:
def CO2_emissions_inlet_known_volume_mole_basis(HCin, FUEL_EFFICIENCY):
    '''
    This function calculates the Carbon dioxide emission rate in tonnes when the flare inlet data is known.
    
    Definition of variables:
    HCin = Flare hydrocarbon inlet rate (to the flare) in SCF
    CFhc = Carbon weight fraction in the inlet hydrocarbon
    FE = Flare combustion efficiency
    Mco2 = Mass of CO2 in flared stream based on CO2 composition of the inlet stream.
    
    CALCULATION STEPS:
    '''
    CO2_mass_emission_rate = convert_scf_to_lbmole(HCin)*(carbon_content_fuel_mixture_mole_basis()*FUEL_EFFICIENCY + mass_CO2_mole_composition_basis(compound, mole_fraction))*convert_lbmole_CO2_to_lbCO2()*convert_lb_to_tonnes()
    return CO2_mass_emission_rate

In [ ]:
def CO2_emissions_inlet_known_mass_weight_fraction_basis(HCin, FUEL_EFFICIENCY):
    '''
    This function calculates the Carbon dioxide emission rate in tonnes when the flare inlet data is known.
    
    Definition of variables:
    HCin = Flare hydrocarbon inlet rate (to the flare) in lb
    FUEL_EFFICIENCY = Flare combustion efficiency
    
    CALCULATION STEPS:
    '''
    CO2_mass_emission_rate = HCin*(carbon_content_fuel_mixture_weight_basis()*FUEL_EFFICIENCY*44/12 + mass_CO2_mole_composition_basis(compound, weight_fraction))*convert_lb_to_tonnes()
    return CO2_mass_emission_rate

In [ ]:
def CO2_emissions_outlet_known_volume_mole_basis(HCout, FUEL_EFFICIENCY):
    '''
    This function calculates the Carbon dioxide emission rate in tonnes when the flare outlet data is known.
    
    Definition of variables:
    HCout = Flare hydrocarbon outlet rate (to the flare) in SCF
    FUEL_EFFICIENCY = Flare combustion efficiency
    
    CALCULATION STEPS:
    '''
    CO2_mass_emission_rate = convert_scf_to_lbmole(HCout)*(carbon_content_fuel_mixture_mole_basis()*(FUEL_EFFICIENCY/(1-FUEL_EFFICIENCY)) + mass_CO2_mole_composition_basis(compound, mole_fraction))*convert_lbmole_CO2_to_lbCO2()*convert_lb_to_tonnes()
    return CO2_mass_emission_rate

In [ ]:
def CO2_emissions_outlet_known_mass_weight_fraction_basis(HCout, FUEL_EFFICIENCY):
    '''
    This function calculates the Carbon dioxide emission rate in tonnes when the flare outlet data is known.
    
    Definition of variables:
    HCout = Flare hydrocarbon outlet rate (to the flare) in lb
    FUEL_EFFICIENCY = Flare combustion efficiency
    
    CALCULATION STEPS:
    '''
    CO2_mass_emission_rate = HCout*(carbon_content_fuel_mixture_weight_basis()*(FUEL_EFFICIENCY/(1-FUEL_EFFICIENCY))*44/12 + mass_CO2_mole_composition_basis(compound, weight_fraction))*convert_lb_to_tonnes()
    return CO2_mass_emission_rate

In [7]:
'''
Pint is a Python package to define, operate and manipulate physical quantities: 
the product of a numerical value and a unit of measurement. 
It allows arithmetic operations between them and conversions from and to different units.

Install pint using:
conda install -c conda-forge pint
or 
pip install pint
'''
import pint
ureg = pint.UnitRegistry()
Q_ = ureg.Quantity

In [8]:
sum([1,2])

3